# High Performance Python

### Activity 3


### Part 3  Matrix and Vector Computations

13. Read about Broadcasting with Arrays on the chapter Computation on Arrays:

Broadcasting from Python Data Science Handbook (J. VandePlas, 2016):

Link: https://jakevdp.github.io/PythonDataScienceHandbook/02.05-computationon-arrays-broadcasting.html.

14. Read Rewriting the particle simulator in Numpy on Chapter 2: Fast Array Operations
with Numpy and Pandas (pp. 68) from the book G. Lenaro (2017). Python high
Performance. Second Edition. UK: Packt Publishing Ltd. Implement the
improvements on the particle simulator using NumPy. Show that both
implementations scale linearly with particle size, but the runtime in the pure Python
version grows much faster than the NumPy version. 

In [19]:
# Libraries
import numpy as np
from random import uniform
from timeit import timeit
import numexpr as ne

In [21]:
class Particle:
    def __init__(self, x, y, velocidad_angular):
        self.x = x
        self.y = y
        self.velocidad_angular = velocidad_angular

class ParticleSimulator:

    def __init__(self, particles):
        self.particles = particles

    def evolve_python(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)
        for i in range(nsteps):
            for p in self.particles:
                norm = (p.x**2 + p.y**2)**0.5
                v_x = (-p.y) / norm
                v_y = p.x / norm
                d_x = timestep * p.velocidad_angular * v_x
                d_y = timestep * p.velocidad_angular * v_y
                p.x += d_x
                p.y += d_y

    def evolve_numpy(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)
        r_i = np.array([[p.x, p.y] for p in self.particles])
        velocidad_angular_i = np.array([p.velocidad_angular for p in self.particles])
        for i in range(nsteps):
            norm_i = np.sqrt((r_i ** 2).sum(axis=1))
            v_i = r_i[:, [1, 0]] * np.array([-1, 1]) / norm_i[:, np.newaxis]
            d_i = timestep * velocidad_angular_i[:, np.newaxis] * v_i
            r_i += d_i
        for i, p in enumerate(self.particles):
            p.x, p.y = r_i[i]

 
def benchmark(npart=1000, method='python'):
    particles = [Particle(uniform(-1.0, 1.0), uniform(-1.0, 1.0), uniform(-1.0, 1.0)) for _ in range(npart)]
    simulator = ParticleSimulator(particles)
    if method == 'python':
        simulator.evolve_python(0.1)
    elif method == 'numpy':
        simulator.evolve_numpy(0.1)

 
# Example of benchmarking
if __name__ == "__main__":

    print("Benchmarking with 100000 particles:")
    print("Pure python:")
    print(timeit("benchmark(1000, 'python')", globals=globals(), number=1))
    print("Numpy:")
    print(timeit("benchmark(1000, 'numpy')", globals=globals(), number=1))



Benchmarking with 100000 particles:
Pure python:
5.024624400000903
Numpy:
0.3494847000001755


15. Explain how to optain the optimal performance with numexpr. Read the section
Reaching optimal performance with numexpr, pp. 72 from the previous reference.
Implement it and measure the execution time.

In [26]:
class Particle:
    def __init__(self, x, y, velocidad_angular):
        self.x = x
        self.y = y
        self.velocidad_angular = velocidad_angular

class ParticleSimulator:

    def __init__(self, particles):
        self.particles = particles

    def evolve_python(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)
        for i in range(nsteps):
            for p in self.particles:
                norm = (p.x**2 + p.y**2)**0.5
                v_x = (-p.y) / norm
                v_y = p.x / norm
                d_x = timestep * p.velocidad_angular * v_x
                d_y = timestep * p.velocidad_angular * v_y
                p.x += d_x
                p.y += d_y

    def evolve_numpy(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)
        r_i = np.array([[p.x, p.y] for p in self.particles])
        velocidad_angular_i = np.array([p.velocidad_angular for p in self.particles])
        for i in range(nsteps):
            norm_i = np.sqrt((r_i ** 2).sum(axis=1))
            v_i = r_i[:, [1, 0]] * np.array([-1, 1]) / norm_i[:, np.newaxis]
            d_i = timestep * velocidad_angular_i[:, np.newaxis] * v_i
            r_i += d_i
        for i, p in enumerate(self.particles):
            p.x, p.y = r_i[i]

    def evolve_numexpr(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)
        r_i = np.array([[p.x, p.y] for p in self.particles])
        velocidad_angular_i = np.array([p.velocidad_angular for p in self.particles])

        for i in range(nsteps):
            # Use numexpr to calculate the norm. We first square r_i components, sum them, and then take the square root.
            # However, we must sum the components outside of numexpr due to the mentioned limitation.
            squared = ne.evaluate("r_i**2")
            sum_squares = np.sum(squared, axis=1)
            norm_i = ne.evaluate("sqrt(sum_squares)"

            v_i = r_i[:, [1, 0]] * np.array([-1, 1])
            v_i = ne.evaluate("v_i / norm_i[:, None]")
            d_i = ne.evaluate("timestep * velocidad_angular_i[:, None] * v_i")
            r_i = ne.evaluate("r_i + d_i")

        for i, p in enumerate(self.particles):
            p.x, p.y = r_i[i]


def benchmark(npart=1000, method='python'):
    particles = [Particle(uniform(-1.0, 1.0), uniform(-1.0, 1.0), uniform(-1.0, 1.0)) for _ in range(npart)]
    simulator = ParticleSimulator(particles)
    if method == 'python':
        start_time = timeit()
        simulator.evolve_python(0.1)
        end_time = timeit()
    elif method == 'numpy':
        start_time = timeit()
        simulator.evolve_numpy(0.1)
        end_time = timeit()
    elif method == 'numexpr':
        start_time = timeit()
        simulator.evolve_numexpr(0.1)
        end_time = timeit()
    return end_time - start_time

# Example of benchmarking
if __name__ == "__main__":
    npart = 1000
    print(f"Benchmarking with {npart} particles:")
    for method in ['python', 'numpy', 'numexpr']:
        time_taken = benchmark(npart, method)
        print(f"{method.capitalize()}: {time_taken} seconds")

Benchmarking with 1000 particles:
Python: 4.70000013592653e-05 seconds
Numpy: -0.0010194999995292164 seconds


UnboundLocalError: cannot access local variable 'v_i' where it is not associated with a value